# ルービンの因果モデル

**ルービンの因果モデル**（rubin causal model）は潜在的結果と割当のメカニズムに着目して因果推論を検討するアプローチ。


### 潜在的結果

ある個体が「介入を受けた場合」と「介入を受けなかった場合」の結果を、実際に介入があったかどうかに関わらず定めたものを**潜在的結果**（potential outcome）という。

- **対照群**（control group）：介入を受けなかった（$D=0$の）場合
- **処置群**（treatment group）：介入を受けた（$D=1$の）場合
- 個体$i$が介入を受けなかった時の潜在的結果：$Y_i(0)$
- 個体$i$が介入を受けた時の潜在的結果：$Y_i(1)$


実際の観察結果は$Y_i$とする

$$
Y_i = \left\{
\begin{array}{l}
Y_i(0) \ \text{if } D_i = 0 \\
Y_i(1) \ \text{if } D_i=1
\end{array}
\right.
$$

次のように書くこともできる

$$
Y_i = Y_i(D_i) = Y_i(0)(1-D_i) + Y_i(1)D
$$


### 因果効果の定義

因果の定義の仕方はいろいろあるが、ルービン因果モデルにおいては「ある個体が介入$D$を受けた場合に、受けなかった場合の結果$Y$と比べた結果$Y_1$の差$Y_1 - Y_0$」を「$D$の**因果効果**（causal effect）」とする。なお、因果効果と同様の意味で**処置効果**（treatment effect）という言葉も使われる。

個体$i$の因果効果（**個体処置効果** individual treatment effect: ITE） は

$$
\tau_i = Y_i(1)-Y_i(0)
$$

となる。

ただし、実際には実行された処置についての結果しか観測することができないため、$\tau_i$は観測することはできない。このことは**因果推論の根本問題**と呼ばれる。

#### 平均処置効果

ITEの推定は難しいため、通常は平均での処置効果を推定することを考える

$$
\tau = E[Y(1) - Y(0)] = E[Y(1)] - E[Y(0)]
$$

これを**平均処置効果**（average treatment effect: ATE）という。


## ATEの推定

### 無作為割り当ての場合

もし処置への割当$D$が無作為（ランダム）であれば潜在的結果$Y(0), Y(1)$とは独立

$$
\newcommand{\indep}{\mathop{\hspace{0.1em} \perp\!\!\!\!\!\perp \hspace{0.1em}}}
(Y(0), Y(1)) \indep D
$$


となり

$$
E[Y(0)| D = 0] = E[Y(0)]\\
E[Y(1)| D = 1] = E[Y(1)]
$$

になるため、実現して観測できる各群の差

$$
\frac{1}{N_1} \sum^{N_1}_{i: D_i = 1} y_i
-
\frac{1}{N_0} \sum^{N_0}_{i: D_i = 0} y_i
$$

がATEの推定量となる。

:::{marginal} 独立のおさらい

2つの事象AとBが独立であるとき、Aが起こる確率はBが起こったという事象とは関係ないため、

$$
P(A|B) = P(A)
$$

となる。


条件付き期待値も同様に

$$
\begin{align}
E[A|B]
&= \int A \times P(A|B) dA\\
&= \int A \times P(A) dA = E[A]
\end{align}
$$

となる
:::


:::{admonition} 例：A/Bテスト
:dropdown:

ECサイトにおいて、10%オフのクーポンをユーザーに対してランダムに配布した場合を仮定する。

クーポンの配布という処置の売上への平均処置効果は、ランダムに割り当てているため、単純に各群の平均の差で計算することができる。


In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ate = 500
n = 100
np.random.seed(0)
d = np.random.binomial(n=1, p=0.5, size=n)
y = np.random.normal(loc=1000, scale=100, size=n) + ate * d

print(f"""
全体の平均：{y.mean():.1f}
対照群の平均：{y[d == 0].mean():.1f}
処置群の平均：{y[d == 1].mean():.1f}
ATEの推定値：{y[d == 1].mean() - y[d == 0].mean():.1f}
""")


全体の平均：1264.2
対照群の平均：1024.5
処置群の平均：1513.8
ATEの推定値：489.3



:::

### 条件付き独立を仮定する場合

共変量$\boldsymbol{X}$を条件付けた下での処置割当$D$と結果$Y(0), Y(1)$の独立

$$
\newcommand{\indep}{\mathop{\hspace{0.1em} \perp\!\!\!\!\!\perp \hspace{0.1em}}}
(Y(0), Y(1)) \indep D | \boldsymbol{X}
$$

を**条件付き独立**（conditional independence）あるいは**強く無視できる割り当て**（strongly ignorable treatment assignment）条件という。

これが仮定できるとき、


## SUTVA条件

個体$i$の潜在的結果$Y_i(1), Y_i(0)$が次の2つの条件を満たすとき、**SUTVA**（stable unit treatment value assumption）条件を満たすという（Rubin 1980）。

1. 個体$i$の潜在的結果は、他の個体が受ける処置の値に依存しない
2. 個体$i$に対する処置は1通りに定まる


### 条件1について

条件1は相互干渉がない（no interference）という言い方もされる（Cox 1958）。

相互干渉がある場合、個体$i$の潜在的結果は個体$i$の処置$D_i$だけでは定まらず、$Y_i(D_1, \cdots, D_n)$のように書く必要が出てきてかなり複雑になる。
そのため、条件1は潜在的結果をシンプルに扱うための条件となる。

しかし現実には相互干渉が発生しうる。例えば労働者への教育プログラムの効果を推定する際には、他にもそのプログラムを受けて技能を向上させた労働者が多く現れる場合、労働市場における希少価値は低下し、結果として個体$i$の潜在的結果に影響を与えると考えられる。

### 条件2について

条件2は処置の内容を明確にすることを要請している。

例えば「ダイエットをする」という処置では、具体的に何をしたのかが人によって大きく異なる可能性がある。

